In [524]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split , GridSearchCV


In [525]:
df = pd.read_csv('https://raw.githubusercontent.com/doeungim/ADP_DE/refs/heads/main/ADP_SET2/data/nba.csv')

df.dtypes

TEAM_NAME          object
PLAYER_ID          object
POSITION_GROUP     object
GAME_ID             int64
SHOT_MADE            bool
ACTION_TYPE        object
SHOT_TYPE          object
ZONE_ABB           object
LOC_X             float64
LOC_Y             float64
QUARTER             int64
MINS_LEFT           int64
SECS_LEFT           int64
dtype: object

* hot_distance (슛거리) : LOC_X, LOX_Y 좌표의 제곱합에 루트를 씌운다
* net_play_time (경기경과시간) : 슛을 시도한 시점의 순 경기 시간을 초(sec)단위로 구한다. 쉬는시간은 고려하지 않는다. 4쿼터 종료시점(MINS_LEFT =0, SECS_LEFT=0)에 슛을 시도했다면 2880 값을 가져야 한다.
* net_score : 해당 게임에 해당팀이 슛을 쏜 결과를 반영한 누적 득점

In [526]:
df['hot_distance'] = np.sqrt(df['LOC_X'] ** 2 + df['LOC_Y'] **2 )

# 1 쿼터 * 12분 * 60 초 
df['net_play_time'] = df['QUARTER'] * 12 * 60 -df['MINS_LEFT'] * 60 - df['SECS_LEFT']
# net_score 
df['SHOT_TYPE'] = df['SHOT_TYPE'].str.split('PT').str[0].astype('int')


dic= {True : 1 , False :0}
df['SHOT_MADE'] = df['SHOT_MADE'].map(dic)


df['score'] = df['SHOT_TYPE'] * df['SHOT_MADE']

In [527]:
df1 = df.copy()

In [528]:
lst = []

for name, game in df1[['TEAM_NAME','GAME_ID']].drop_duplicates().values :  # ['team_5', 22300003]
    target = df1[(df1.TEAM_NAME == name) & (df1.GAME_ID == game)]
    target['net_score'] = target['score'].cumsum()

    lst.append(target)


p_df = pd.concat(lst).reset_index(drop = True)

In [529]:
# 그룹by 풀이 : 
#df2 =df1.copy()a
#df2['cum'] =df2[['GAME_ID','TEAM_NAME','PLAYER_ID','net_play_time', 'score']].groupby(['GAME_ID','TEAM_NAME'])['score'].cumsum()

In [530]:
# 각 행은 하나의 팀, 하나의 게임 : groupby

sc = p_df.groupby(['GAME_ID' ,'TEAM_NAME'])['net_score'].max().reset_index()

lst =[]
for id in sc.GAME_ID.unique() :
    target = sc[sc.GAME_ID == id].sort_values('net_score').reset_index(drop = True)
    target['result'] = [0,1]
    lst.append(target)

result_df = pd.concat(lst).reset_index(drop = True)
result_df.head(10)

,GAME_ID,TEAM_NAME,net_score,result
0,22300001,team_2,96,0
1,22300001,team_1,105,1
2,22300002,team_3,86,0
3,22300002,team_4,90,1
4,22300003,team_5,105,0
5,22300003,team_6,109,1
6,22300004,team_7,97,0
7,22300004,team_8,106,1
8,22300005,team_9,115,0
9,22300005,team_10,116,1


In [531]:
lst = []
for team, game in p_df[['TEAM_NAME','GAME_ID']].drop_duplicates().values:    
    
    data = []
    target = p_df[(p_df.TEAM_NAME ==team) & (p_df.GAME_ID ==game)].reset_index(drop=True)
    data += [team,game]
    
    # play_time 기준 
    
    for play_time in range(360,360*6+1,360):
        filter_df = target[(target.net_play_time <= play_time) & (target.net_play_time > (play_time -360))].sort_values('net_play_time')
        
        ratio_2pt = filter_df[(filter_df.SHOT_TYPE ==2) & (filter_df.SHOT_MADE ==False)].shape[0] 
        ratio_3pt = filter_df[(filter_df.SHOT_TYPE ==3) & (filter_df.SHOT_MADE ==False)].shape[0] 
        data +=[ratio_2pt,ratio_3pt]
        
    quarter_1_score = target[target.QUARTER ==1].net_score.max()
    quarter_3_score = target[target.QUARTER ==3].net_score.max()
        
    data +=[quarter_1_score,quarter_3_score]
    
    lst.append(data)

## col_name 지정 
col_name = []
for q in range(1,4) : 
    for s in range(1,3): 
        for score in range(2,4,) : 
            col_name.append(f'{q}_{s}_try{s}')

## col_name 프레임에 씌우기
col_names = ['TEAM_NAME','GAME_ID'] + col_name + ['q1_score', 'q3_score']
second = pd.DataFrame(lst, columns = col_names)

In [532]:
df_pre = pd.merge(result_df,second).drop(columns =['net_score'])
df_pre

,GAME_ID,TEAM_NAME,result,1_1_try1,1_1_try1,1_2_try2,1_2_try2,2_1_try1,2_1_try1,2_2_try2,2_2_try2,3_1_try1,3_1_try1,3_2_try2,3_2_try2,q1_score,q3_score
0,22300001,team_2,0,5,2,2,3,3,2,1,5,3,1,1,1,24,76
1,22300001,team_1,1,5,2,3,1,3,1,2,3,4,3,4,3,33,79
2,22300002,team_3,0,7,3,7,3,0,5,7,3,0,6,0,5,24,62
3,22300002,team_4,1,1,1,6,4,4,3,4,0,3,3,4,3,20,74
4,22300003,team_5,0,1,0,1,2,6,1,2,4,3,3,1,1,33,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,22301228,team_27,1,3,7,1,3,4,4,6,3,4,1,2,1,22,76
2338,22301229,team_4,0,4,1,6,3,6,1,2,3,0,3,7,1,25,78
2339,22301229,team_1,1,2,2,6,3,5,1,0,6,2,6,1,3,25,80
2340,22301230,team_19,0,5,2,5,3,6,1,0,4,5,4,5,3,28,63


game_id가 홀수로 끝나는 경우는 승리팀만, 짝수를 끝나는 경우는 패배팀만 필터한다.
랜덤포레스트와 xgb를 사용하여 모델링을 진행하라. 데이터의 층화 추출을 통해 70%의 데이터로 학습하고 30% 데이터로 검증

In [533]:
filter1 = df_pre[(df_pre['GAME_ID']%2 == 0) & (df_pre['result'] == 0)]
filter2 = df_pre[(df_pre['GAME_ID']%2 != 0) & (df_pre['result'] == 1)]

train_df = pd.concat([filter1, filter2]).reset_index(drop = True)

In [542]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

y = train_df['result']
X = train_df.drop(['GAME_ID','TEAM_NAME','result'], axis =1)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify= y , random_state = 1234)


In [543]:
## numpy로 변환 
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()

rf = RandomForestClassifier(n_estimators=100, random_state=42)
xg = xgb.XGBClassifier(random_state = 123)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train_np, y_train)
rf_predict = rf.predict(x_test_np)

xg = xgb.XGBClassifier(random_state=123)
xg.fit(x_train_np, y_train)
xg_predict = xg.predict(x_test_np)

rf_accuracy = accuracy_score(y_test, rf_predict)
xgb_accuracy = accuracy_score(y_test, xg_predict)
print('rf : ',rf_accuracy,'xgb :',xgb_accuracy)

rf :  0.6306818181818182 xgb : 0.6221590909090909


In [565]:
from sklearn.datasets import fetch_california_housing
cali = fetch_california_housing()
X,y=pd.DataFrame(cali.data,columns =cali['feature_names']), pd.DataFrame(cali.target,columns =['MedHouseVal'])


In [564]:
## 변수들의 다중 공선성을 검토 
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_df= pd.DataFrame()
vif_df['columns'] = X.columns
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_df

,columns,VIF
0,MedInc,11.511140
1,HouseAge,7.195917
2,AveRooms,45.993601
3,AveBedrms,43.590314
4,Population,2.935745
5,AveOccup,1.095243
6,Latitude,559.874071
7,Longitude,633.711654


In [568]:
## 회귀모델을 제작할 때 다중공선성을 줄이기 위한 3가지 방법을 서술하라. 

# 1. 변수 제거 : 다중 공선성이 있는 변수 중 일부를 제거 
# 2. 주성분 분석 - 주성분으로 변환 후, 상위 몇 개의 주성분만 사용하여 회귀 모델을 구성
# 3. 정규화 회귀 모델 사용 : (Ridge, Lasso, ElasticNet)


## 7:3으로 분리 후, RMSE 기준 평가 , Elastic 모델의 경우, L1, L2 

from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA 
from sklearn.linear_model import LinearRegression , ElasticNet


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3 , random_state = 42)

lr = LinearRegression()
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
print(lr_rmse)

0.7284008391515455


In [ ]:
# alpha : 정규화 강도 - 패털티의 강도를 의미 계수를 더 줄인다. 
# l1 : Lasso (특정 변수의 계수를 0으로) , L2 : Ridge (모든 변수의 계수를 줄이고)
ElasticNet()

In [571]:
alpha= [0.2, 0.5,1.0]
l1_ratio=[0.2,0.5,0.8] 

lst = []
for a in alpha : 
    for l1 in l1_ratio : 
        en = ElasticNet(alpha = a, l1_ratio = l1, random_state = 42)
        en.fit(x_train, y_train)
        en_pred = en.predict(x_test)
        rmse = np.sqrt(mean_squared_error(y_test, en_pred))

        lst.append([a, l1, rmse])

elastic_result = pd.DataFrame(lst , columns = ['Alpha','L1_ratio', 'RMSE'])
elastic_result

,Alpha,L1_ratio,RMSE
0,0.2,0.2,0.752719
1,0.2,0.5,0.779761
2,0.2,0.8,0.803567
3,0.5,0.2,0.794009
4,0.5,0.5,0.821227
5,0.5,0.8,0.834741
6,1.0,0.2,0.833016
7,1.0,0.5,0.869651
8,1.0,0.8,0.920629


##통계1
* 한 제조회사에서는 제품의 품질을 보장하기 위해 생산 라인에서 무작위로 10개의 제품을 샘플링하여 품질 검사를 실시한다. 이때, 제품이 불량일 확률은 23%다.
- (1) 적어도 3개의 제품이 불량일 확률은? 
- (2) 2개 이하의 제품이 불량일 확률은?

In [574]:
from scipy.stats import binom 

rv = binom(10, 0.23)

# 1) 3개 이상 1 - cdf(2)
print(1 - rv.cdf(2))

# 2) 2개 이하 
print(rv.cdf(2))

0.4137172738381183
0.5862827261618817


어느 고객 서비스 센터에는 센터에는 평균적으로 1분에 3건의 전화가 걸려온다. 
전화 빈도는 아래와 같은 조건을 가진다. 전화가 걸려오는 사건이 독립적이다. 
특정 시간 동안에 전화가 걸려올 확률이 일정하다. 단위 시간 내에 동시에 두 개 이상의 전화가 걸려올 확률은 매우 낮다. </br>

(1) 10분 동안 전화가 23통 이하로 걸려올 확률은? </br>
(2) 10분 동안 전화가 정확히 30통 걸려올 확률은?

In [586]:
from scipy.stats import poisson , norm

# 1분에 3건 :30 
lamb = 30

#1) 23이하 
print(poisson.cdf(23, lamb))

#2)  정확히 30통 걸려올 확률
print(poisson.pmf(30, lamb))

0.11464591271427384
0.07263452647159181


통계 3
한 회사에서 직원들의 근무 만족도가 개선되었는지를 평가하기 위해, </br>
직원들에게 개선 전과 개선 후의 만족도를 각각 설문조사했다.</br>
만족도 점수는 1에서 10까지의 범위로 측정된다. 다음은 10명의 직원이 제공한 만족도 점수다 </br>

개선 전: [5, 6, 7, 5, 6, 8, 7, 5, 6, 9] </br>
개선 후: [6, 7, 8, 6, 7, 9, 8, 6, 7, 10]

In [582]:
from scipy.stats import wilcoxon

# 대응 비모수 wilconx 
before =  [5, 6, 7, 5, 6, 8, 7, 5, 6, 9]
after = [6, 7, 8, 6, 7, 9, 8, 6, 7, 10]

stat, p_val = wilcoxon(before, after)
p_val

np.float64(0.001953125)

183개의 샘플을 추출했을 때 샘플 무게의 평균이 69.5g에서 70.1g 사이에 있을 확률

In [584]:
df = pd.read_csv('https://raw.githubusercontent.com/doeungim/ADP_DE/refs/heads/main/ADP_SET2/data/s5.csv')
df.head()

,w
0,78.8
1,72.0
2,74.9
3,81.2
4,79.3


In [587]:
mean = df['w'].mean()
std = df['w'].std()

sample = 183 

# SE 
se = std / np.sqrt(sample)
print(norm.cdf(70.1, loc = mean, scale = se) - norm.cdf(69.5, loc = mean, scale = se))

0.5876910371573985
